In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import tensorflow as tf
tf.test.gpu_device_name()

Mounted at /content/drive


'/device:GPU:0'

In [ ]:
!pip install transformers
!pip install --upgrade accelerate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset
import datetime
import numpy as np
import random
import torch.nn as nn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.3 MB/s eta 0:00:00


## Preparations

In [ ]:
import pickle
#We use the data we processed in previous steps
Data = pd.read_csv("/content/drive/MyDrive/DeepLearning - Headlines Proj/PreprocessedData0506.csv")
Data['preprocessed_headline_text'] = Data['preprocessed_headline_text'].fillna('<missing>').astype(str)

# Assume your Data variable is a pandas DataFrame
preprocessed_headlines = Data['preprocessed_headline_text'].tolist()

# Encode the labels with the same encoder you used during training
#encoder = LabelEncoder()
label_dict = {"business": 0, "culture": 1, "lifestyle": 2, "news": 3, "opinion": 4, "sport": 5}

# Transform the labels
labels = Data['primary_category'].apply(lambda x: label_dict[x]).values.tolist()


# Tokenize the preprocessed_headlines
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#encoded_data = tokenizer.batch_encode_plus(preprocessed_headlines, truncation=True, padding=True, max_length=15)
# Load the variable
with open('/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/encoded_data.pkl', 'rb') as file:
    encoded_data = pickle.load(file)

input_ids = torch.tensor(encoded_data["input_ids"])
attention_masks = torch.tensor(encoded_data["attention_mask"])

#original_dataset = TensorDataset(input_ids, attention_masks, torch.tensor(labels))
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Construct a CustomDataset
original_dataset = CustomDataset(encoded_data, labels)


# Compute metrics function for validation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



In [ ]:
import pickle

# Save the variable
with open("/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/encoded_data.pkl", 'wb') as file:
    pickle.dump(encoded_data, file)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DeepLearning - Headlines Proj/files/train_df_FULL.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DeepLearning - Headlines Proj/files/test_df_FULL.csv')
train_distilbert_padded = np.genfromtxt('/content/drive/MyDrive/DeepLearning - Headlines Proj/files/train_distilbert_padded_FULL.csv', delimiter=",")
test_distilbert_padded = np.genfromtxt('/content/drive/MyDrive/DeepLearning - Headlines Proj/files/test_distilbert_padded_FULL.csv', delimiter=",")

In [ ]:
#we use the chosen checkpoint as the model to compress
checkpoint_220K_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-220000"

# Load the model
model = DistilBertForSequenceClassification.from_pretrained(checkpoint_220K_path)

## Compressing by Distillation Knowledge

In [ ]:
from torch.nn.utils import prune
import torch.quantization
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
# Instantiate a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/results_2nd_distillation',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    save_steps=7000,
)

class NewsHeadlinesDataset(Dataset):
    def __init__(self, tokenized_inputs, labels):
      self.tokenized_inputs = tokenized_inputs  # Tokenized and padded input sequences
      self.labels = labels  # Encoded labels (categories)

    def __len__(self):
      return len(self.labels)  # Return the number of samples

    def __getitem__(self, idx):
      return {'input_ids': torch.tensor(self.tokenized_inputs[idx], dtype=torch.long).to(device),  # Convert input sequence to tensor
              'labels': torch.tensor(self.labels[idx], dtype=torch.long).to(device),  # Convert label to tensor
              }


# Convert the input data and labels to PyTorch tensors using the custom dataset class
train_labels = train_df['encoded_category'].values
test_labels = test_df['encoded_category'].values

train_distilbert_dataset = NewsHeadlinesDataset(train_distilbert_padded, train_labels)
test_distilbert_dataset = NewsHeadlinesDataset(test_distilbert_padded, test_labels)

from torch.nn import KLDivLoss
import torch.nn.functional as F


class DistillationTrainer(Trainer):
    def __init__(self, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.temperature = 1.0  # Add this line

    def compute_loss(self, model, inputs, return_outputs=False):
      labels = inputs.pop("labels")
      teacher_model_output = teacher_model(**inputs)
      student_model_output = model(**inputs)
      loss_fct = KLDivLoss(reduction='batchmean')
      loss = loss_fct(
          F.log_softmax(student_model_output.logits / self.temperature, dim=-1),
          F.softmax(teacher_model_output.logits / self.temperature, dim=-1),
        )
      return (loss, student_model_output) if return_outputs else loss

#the teacher model is our trained model and the student is a regular distilbert model
teacher_model = DistilBertForSequenceClassification.from_pretrained(checkpoint_220K_path)
student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

distillation_trainer = DistillationTrainer(
    model=student_model,
    args=training_args,
    train_dataset=train_distilbert_dataset,
    eval_dataset=test_distilbert_dataset,
    data_collator=data_collator,
)

# check if GPU is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("device is:", device)
# move models to the correct device
teacher_model.to(device)
student_model.to(device)

distillation_trainer.train()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

Step,Training Loss
500,0.724700
1000,0.573500
1500,0.539800
2000,0.530900
2500,0.494700
3000,0.479800
3500,0.461100
4000,0.462100
4500,0.454400
5000,0.435600


TrainOutput(global_step=56130, training_loss=0.3438627229789436, metrics={'train_runtime': 4258.0638, 'train_samples_per_second': 210.913, 'train_steps_per_second': 13.182, 'total_flos': 3485590071292800.0, 'train_loss': 0.3438627229789436, 'epoch': 1.0})

## Compressing by pruning 

In [ ]:
from torch.nn.utils import prune
import torch.quantization
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Assume that your model is loaded here
#model = DistilBertForSequenceClassification.from_pretrained(checkpoint_220K_path)
model.eval()

# Pruning
for module in model.modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name="weight", amount=0.2)
        prune.remove(module, 'weight')

pruned_model = model
torch.save(pruned_model.state_dict(), "/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/pruned_model.pth")

## compressing by low rank approximation

In [ ]:
import torch

def low_rank_approximation(model, rank):
    for name, param in model.named_parameters():
        if param.requires_grad and len(param.shape) == 2:
            u, s, v = torch.svd(param.data)
            u_approx = u[:, :rank]
            s_approx = s[:rank]
            v_approx = v[:, :rank]
            param.data = u_approx @ torch.diag(s_approx) @ v_approx.T
    return model


In [ ]:
from torch.utils.data import random_split

# Define the split size
val_size = int(0.5 * len(original_dataset))  # 20% for validation
train_size = len(original_dataset) - val_size

# Split the dataset
train_dataset, val_dataset = random_split(original_dataset, [train_size, val_size])

### finding the optimal rank to use

In [ ]:
from transformers import Trainer, TrainingArguments

ranks = [50,75,100]

# We don't need TrainingArguments for evaluation, but it might be useful if you want to customize the evaluation process
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': precision_recall_fscore_support(labels, predictions, average='weighted')[2]
    }

for rank in ranks:
    # Applying the approximation
    approx_model = low_rank_approximation(model, rank)

    # Transfer the tensors in the evaluation dataset to the CPU
    eval_dataset_cpu = []
    for example in test_distilbert_dataset:
        example_cpu = {k: v.cpu() for k, v in example.items()}
        eval_dataset_cpu.append(example_cpu)

    # Initializing the trainer
    trainer = Trainer(model=approx_model, eval_dataset=eval_dataset_cpu, compute_metrics=compute_metrics)

    # Evaluation
    eval_result = trainer.evaluate()

    # Print evaluation results
    print(f"Evaluation results for rank {rank}: ", eval_result)

    # Save the model
    #model_path = f"/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/low_rank_approximation/approx_model_rank_{rank}.pth"
    #torch.save(approx_model.state_dict(), model_path)


Evaluation results for rank 50:  {'eval_loss': 2.5576417446136475, 'eval_accuracy': 0.14715392838054517, 'eval_f1': 0.11532256561258057, 'eval_runtime': 227.3094, 'eval_samples_per_second': 987.729, 'eval_steps_per_second': 123.466}


Evaluation results for rank 75:  {'eval_loss': 2.557910442352295, 'eval_accuracy': 0.14722964546588277, 'eval_f1': 0.11536732126985356, 'eval_runtime': 228.0311, 'eval_samples_per_second': 984.603, 'eval_steps_per_second': 123.075}


Evaluation results for rank 100:  {'eval_loss': 2.558204174041748, 'eval_accuracy': 0.14722964546588277, 'eval_f1': 0.1154047342259352, 'eval_runtime': 229.6371, 'eval_samples_per_second': 977.717, 'eval_steps_per_second': 122.215}


In [ ]:
compressed_model = low_rank_approximation(model,100)
model_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/final models/approx_model_rank_100.pth"
torch.save(compressed_model.state_dict(), model_path)

### compressing with the chosen rank

In [ ]:
from transformers import DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': precision_recall_fscore_support(labels, predictions, average='weighted')[2]
    }
# Load the original model
#model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-220000', num_labels=6)

# Apply low rank approximation
#compressed_model = low_rank_approximation(model, 30)

# Initialize the trainer
trainer = Trainer(
    model=compressed_model,
    eval_dataset=original_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate on the entire dataset
eval_result = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results for best rank on entire dataset: ", eval_result)

Evaluation results for best rank on entire dataset:  {'eval_loss': 3.195000171661377, 'eval_accuracy': 0.07381753101409734, 'eval_f1': 0.05493152943114579, 'eval_runtime': 1602.6305, 'eval_samples_per_second': 948.663, 'eval_steps_per_second': 118.583}


## Evaluating the 3 compressed models to find the best one

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer
model_path = "/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/final models/pruned_model.pth"

# Instantiate the DistilBERT model and tokenizer
pruned_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load the state dictionary from the saved model file
state_dict = torch.load(model_path)

# Load the state dictionary into the model
pruned_model.load_state_dict(state_dict)
# Set the model to evaluation mode
pruned_model.eval()

# Optional: Move the model to a specific device (e.g., GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pruned_model = pruned_model.to(device)

# Transfer the tensors in the evaluation dataset to the CPU
eval_dataset_cpu = []
for example in test_distilbert_dataset:
    example_cpu = {k: v.cpu() for k, v in example.items()}
    eval_dataset_cpu.append(example_cpu)

trainer_purned = Trainer(
    model=pruned_model,
    eval_dataset=eval_dataset_cpu,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [ ]:
pruned_results = trainer_purned.evaluate()
print(f"Evaluation results for pruned model on validation dataset: ", pruned_results)

Evaluation results for pruned model on validation dataset:  {'eval_loss': 0.8360041975975037, 'eval_accuracy': 0.7610992339212542, 'eval_f1': 0.7596381623763788, 'eval_runtime': 315.1851, 'eval_samples_per_second': 712.343, 'eval_steps_per_second': 89.043}


In [ ]:
full_model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/DeepLearning - Headlines Proj/The trained model/checkpoint-220000', num_labels=6)
trainer_full = Trainer(
    model=full_model,
    eval_dataset=eval_dataset_cpu,
    compute_metrics=compute_metrics,
)
full_results = trainer_full.evaluate()
print(f"Evaluation results for full model on validation dataset: ", full_results)

Evaluation results for full model on validation dataset:  {'eval_loss': 0.8300805687904358, 'eval_accuracy': 0.7602975236059148, 'eval_f1': 0.7589917045485225, 'eval_runtime': 313.66, 'eval_samples_per_second': 715.807, 'eval_steps_per_second': 89.476}


In [ ]:
distiled_model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/DeepLearning - Headlines Proj/compressing/final models/checkpoint-56000', num_labels=6)
trainer_distiled_model = Trainer(
    model=distiled_model,
    eval_dataset=eval_dataset_cpu,
    compute_metrics=compute_metrics,
)
distiled_results = trainer_distiled_model.evaluate()
print(f"Evaluation results for distiled model on validation dataset: ", distiled_results)

Evaluation results for distiled model on validation dataset:  {'eval_loss': 0.735588014125824, 'eval_accuracy': 0.7456306787814003, 'eval_f1': 0.7429408723572842, 'eval_runtime': 241.4879, 'eval_samples_per_second': 929.736, 'eval_steps_per_second': 116.217}
